## Task 2. Heuristic user segmentation

In this task you should firstly parse the user logs. Then distiguish the segments and count the *unique* uids in each segment. Sort the output by counts.

You may find more useful methods in the following sources:

* Book "Learning Spark: Lightning-Fast Big Data Analysis" by Holden Karau.

* [Spark Streaming documentation](https://spark.apache.org/docs/latest/streaming-programming-guide.html)

* [PySpark Streaming documentation](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark-streaming-module) 

* [PySpark Streaming examples](https://github.com/apache/spark/tree/master/examples/src/main/python/streaming)

* [HyperLogLog documentation](https://pypi.org/project/hyperloglog/)

* [Ua_Parser_documentation](https://pypi.org/project/ua-parser/0.7.0/)

In [1]:
import os
import time
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext

# You need also use these specific libraries
from ua_parser import user_agent_parser
from hyperloglog import HyperLogLog

In [2]:
# Here is an example of `user_agent_parser` usage

# ua = 'Mozilla/5.0 (iPad; CPU OS 7_0_4 like Mac OS X) AppleWebKit/537.51.1 (KHTML, like Gecko) Version/7.0 Mobile/11B554a Safari/9537.53'
# user_agent_parser.Parse(ua)

In [3]:
# Here is an example of `hyperloglog` usage

# from hyperloglog import HyperLogLog
# hll = HyperLogLog(0.01)
# for word in ['abc', 'acb', 'def', 'fghijk']:
#     hll.add(word)
# print hll.card(), int(hll.card())

**NB.** Please don't change the cell (even comments) below. It is used for emulation realtime batch arriving. But figure out the code, it will help you when you work with real SparkStreaming applications.

In [4]:
sc = SparkContext(master='local[4]')

# Preparing batches with the input data
DATA_PATH = "/data/course4/uid_ua_100k_splitted_by_5k"
batches = [sc.textFile(os.path.join(DATA_PATH, path)) for path in os.listdir(DATA_PATH)]

# Creating Dstream to emulate realtime data generating
BATCH_TIMEOUT = 5 # Timeout between the batch generation
ssc = StreamingContext(sc, BATCH_TIMEOUT)
dstream = ssc.queueStream(rdds=batches)

There are 2 flags used in this task. 
* The `finished` flag indicates if the current RDD is empty.
* The `printed` one indicates the the result has been printed and SparkStreaming context can be stopped.

**NB**. Spark transformations work in a lazy mode. When the transformation is called, it doesn't execute really. It just saves in the computational DAG. All the transformations will be executed when the action will be called. Let's look at `print_only_at_the_end()` function. The action will be called only when the stream will be finished. So in this moment  Spark will execute all the transformations. This will lead to container's overflow if the dataset is really big. So if you faced the error like `Container killed by YARN for exceeding memory limits`, call some action (e.g. `rdd.count()`) before `if` clause in this function.

In [5]:
finished = False
printed = False

def set_ending_flag(rdd):
    global finished
    if rdd.isEmpty():
        finished = True

def print_only_at_the_end(rdd):
    global printed
    print(rdd.map(lambda x: (x[0],int(x[1].card()))).collect())
    if finished and not printed:
        # Type your code for sorting and printing the resulting RDD
        print("final")
        print(rdd.map(lambda x: (x[0],int(x[1].card()))).collect())
        printed = True

# If we have received empty an rdd, the stream is finished.
# So print the result and stop the context.

dstream.foreachRDD(set_ending_flag)

In [6]:
# Type your code for data processing and aggregation here
baseDF = sc.parallelize([['seg_windows',0],['seg_firefox',0],['seg_iphone',0]])
def agg(new_list,old):
    if not old:
        old = HyperLogLog(0.01)
    for el in new_list:
        old.add(el)
    return old

def classify(line):
    user = line.split("\t")[0]
    ua = line.split("\t")[1]
    parsed_ua = user_agent_parser.Parse(ua)
    ans = []
    if 'iPhone' in parsed_ua['device']['family']:
        ans.append( ('seg_iphone',user))
    if 'Firefox' in parsed_ua['user_agent']['family']:
        ans.append(('seg_firefox',user))
    if 'Windows' in parsed_ua['os']['family']:
        ans.append( ('seg_windows',user))
    return ans
        
#dstream.flatMap(lambda x:classify(x)).updateStateByKey(agg).map(lambda x: (x[0],int(x[1].card()))) \
dstream.map(lambda x:classify(x)).filter(lambda x:len(x)>0).flatMap(lambda x: x) \
    .updateStateByKey(agg) \
    .foreachRDD(print_only_at_the_end)
# dstream.map(...) \
#        .filter(...) ...

In [7]:
classify("abc\t Mozilla/5.0 (iPad; CPU OS 7_0_4 like Mac OS X) AppleWebKit/537.51.1 (KHTML, like Gecko) Version/7.0 Mobile/11B554a Safari/9537.53")

[]

**NB.** Please don't change the cell below. It is used for stopping SparkStreaming context and Spark context when the stream is finished.

In [8]:
ssc.checkpoint('./checkpoint{}'.format(time.strftime("%Y_%m_%d_%H_%M_%s", time.gmtime())))  # checkpoint for storing current state        
ssc.start()
while not printed:
    time.sleep(0.1)
ssc.stop()  # when the result printed, stop SparkStreaming context
sc.stop()  # stop Spark context to be able to restart the code without restarting the kernel

[('seg_firefox', 454), ('seg_iphone', 138), ('seg_windows', 3195)]
[('seg_firefox', 968), ('seg_iphone', 282), ('seg_windows', 6368)]
[('seg_firefox', 1415), ('seg_iphone', 434), ('seg_windows', 9492)]
[('seg_firefox', 1909), ('seg_iphone', 582), ('seg_windows', 12569)]
[('seg_firefox', 2346), ('seg_iphone', 747), ('seg_windows', 15696)]
[('seg_firefox', 2813), ('seg_iphone', 904), ('seg_windows', 18654)]
[('seg_firefox', 3263), ('seg_iphone', 1033), ('seg_windows', 21627)]
[('seg_firefox', 3726), ('seg_iphone', 1168), ('seg_windows', 24529)]
[('seg_firefox', 4169), ('seg_iphone', 1310), ('seg_windows', 27497)]
[('seg_firefox', 4575), ('seg_iphone', 1449), ('seg_windows', 30390)]
[('seg_firefox', 5008), ('seg_iphone', 1601), ('seg_windows', 33122)]
[('seg_firefox', 5427), ('seg_iphone', 1742), ('seg_windows', 35980)]
[('seg_firefox', 5807), ('seg_iphone', 1874), ('seg_windows', 38549)]
[('seg_firefox', 6210), ('seg_iphone', 1996), ('seg_windows', 40926)]
[('seg_firefox', 6576), ('seg_i

Here you can see an output on the sample dataset:
```
seg_windows 24241
seg_firefox 4176
seg_iphone 1361
```
Of course, the numbers may be different but not very much (the error about 10% will be accepted).

Also, remove trailing empty cells before submission.